# Hangman game

## Initialize
- Acquisition of word list
- Random select a word
- Initialize number of trials
- (take into account the word frequency)

## Play
- output for the user
- method for trials
- check

In [1]:
from typing import List
import numpy as np

In [62]:
class Hangman(object):
    
    def __init__(self, 
                 list_of_words: List[str],
                 word_len: int = 6,
                 trials: int = 8):
        self._idx = {}
        self.word_len, self.trials = word_len, trials
        self._update_index(list_of_words)
        self.selected_word = None
        self._select_word()
        self.mask = ['_'] * len(self.selected_word)
                
    def _update_index(self, list_of_words: List[str]):
        for word in list_of_words:
            try:
                self._idx[len(word)].append(word)
            except KeyError:
                self._idx[len(word)] = [word]
    
    def get_words(self, length: int):
        try:
            return self._idx[length]
        except KeyError:
            return []
        
    def add_words(self, list_of_words: List[str]):
        self._update_index(list_of_words)
        
    def print_status(self):
        print(self.mask)
        print('\nRemaining trials {}'.format(self.trials))
        
    def play(self, guess: str):
        self.trials -= 1
        for i, ch in enumerate(self.selected_word):
            if guess[i] == ch:
                self.mask[i] = ch
        return self.mask
         
    def _select_word(self):
        try:
            candidates = self._idx[self.word_len]
            self.selected_word = np.random.choice(candidates)
        except KeyError:
            print(
                'Non esistono parole di lunghezza {}'.format(
                self.word_len))

In [63]:
file_path = 'words/1000_parole_italiane_comuni.txt'
with open(file_path, 'r') as fhandle:
    lines = fhandle.readlines()
words = [w.strip('\n') for w in lines]

In [64]:
h = Hangman(words, word_len=6, trials=10)

In [65]:
h.print_status()

['_', '_', '_', '_', '_', '_']

Remaining trials 10


In [66]:
h.play(guess='_o__ie')

['_', '_', '_', '_', '_', 'e']

## Create an automatic player based on char frequency
- frequenza delle lettere
- provo con stringhe composte sempre dalla stessa lettera
- non modifico la string tentativo (guess) nelle posizioni già indovinate

### Dati che mi servono
- lettere in ordine discendente di frequenza

In [17]:
from collections import defaultdict

In [14]:
file_path = 'words/1000_parole_italiane_comuni.txt'
with open(file_path, 'r') as fhandle:
    lines = fhandle.readlines()
words = [w.strip('\n') for w in lines]

In [100]:
class Player(object):
    
    def __init__(self, hangman: Hangman, list_of_words: List[str]):
        self.hangman = hangman
        char_index = defaultdict(lambda: 0)
        for word in list_of_words:
            for char in word:
                char_index[char] += 1
        self.char_frequency = [c for c, f in sorted(list(
            char_index.items()), key=lambda x: -x[1])]
        self.char_to_try = 0
        self.positions_guessed = {}
    
    def trial(self):
        guess = self.char_frequency[
            self.char_to_try] * self.hangman.word_len
        mask = self.hangman.play(guess)
        self.char_to_try += 1
        for i, ch in enumerate(mask):
            if ch != '_':
                self.positions_guessed[i] = ch
    
    def play(self):
        while True:
            self.trial()
            if self.hangman.trials < 0:
                print('Perso!', self.hangman.mask)
                break
            elif '_' not in self.hangman.mask:
                print('Win!', self.hangman.mask, self.hangman.trials)
                break
            else:
                continue

In [109]:
h = Hangman(words, word_len=6, trials=16)
player = Player(h, words)

In [110]:
player.play()

Win! ['o', 'r', 'a', 'm', 'a', 'i'] 4
